In [ ]:
from utils import load_api_key
API_KEY = load_api_key()

# Tools

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", api_key=API_KEY)

## Tool predefinida por LangChain

In [ ]:
llm.invoke("¿En qué fecha has sido entrenado?").content

In [ ]:
llm.invoke("¿Quien es el presidente de Estados Unidos?")

Acceso a la [documentación de WikipediaApiWrapper](https://python.langchain.com/api_reference/community/utilities/langchain_community.utilities.wikipedia.WikipediaAPIWrapper.html#langchain_community.utilities.wikipedia.WikipediaAPIWrapper)

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(
    lang="es",
    top_k_results=3,
    # doc_content_chars_max=1000
)
wikipedia_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

Vamos a probar a hacer directamente una búsqueda manualmente en wikipedia

In [ ]:
print(f"Name: {wikipedia_tool.name}")
print(f"Description: {wikipedia_tool.description}")
print(f"Args: {wikipedia_tool.args}")
print("----------------------------------------------")

print(wikipedia_tool.invoke({"query": "python"}))

Ahora vamos a unir `wikipedia_tool` con nuestro LLM y así poder buscar automáticamente esta información. 

El orden será:
1. Usar nuestro LLM para escribir los parámetros de la tool
2. Ejecutar la tool (y así añadir la información necesaria al contexto)
3. Volver a llamar al LLM con todo el contexto

In [ ]:
tools = {"wikipedia": wikipedia_tool}

llm_with_tools = llm.bind_tools([t for t in tools.values()])

⚠️ **Nota**: No todos los modelos de LLM tienen la capacidad de ejecutar Tools!

In [ ]:
from langchain_core.messages import HumanMessage
messages = []
prompt = HumanMessage("Quién es el presidente de Estados Unidos?")
messages.append(prompt)

# 1. Generar petición a Tool
ai_msg = llm_with_tools.invoke(messages)
print(f"ContentString: {ai_msg.content}")
print(f"ToolCalls: {ai_msg.tool_calls}")
messages.append(ai_msg)

In [ ]:
# 2. Realizar petición a la Tool y guardar en ChatHistory
if ai_msg.tool_calls:
    for tool_call in ai_msg.tool_calls:
        tool_msg = tools[tool_call["name"]].invoke(tool_call)
        # print(f"ContentString: {tool_msg}")
        messages.append(tool_msg)
display(messages)

In [ ]:
# 3. Repetir llamada, incluyendo salida de Tool
response = llm_with_tools.invoke(messages)
messages.append(response)
response

In [ ]:
for msg in messages:
    msg.pretty_print()

## Tool personal

A continuación vamos a crear una custom Tool. Para ello, vamos a definir una función de Python y la decoraremos con el decorador `@tool`

❗❗ OJO: Para poder crear una Tool a partir de una función de Python debemos proporcionar los typing annotations y un docstring

In [ ]:
from langchain_core.tools import tool

@tool
def multiplicar_numeros(num1: int, num2: int) -> int:
    """
    Multiplies two integers and returns the result.

    Args:
        num1 (int): The first integer.
        num2 (int): The second integer.

    Returns:
        int: The result of multiplying num1 and num2.
    """
    return num1 * num2


In [ ]:
print(f"Name: {multiplicar_numeros.name}")
print(f"Description: {multiplicar_numeros.description}")
print(f"Args: {multiplicar_numeros.args}")
print("----------------------------------------------")

print(multiplicar_numeros.invoke({"num1": 12, "num2": 2}))

In [ ]:
tools = {"wikipedia": wikipedia_tool, "multiplicar_numeros": multiplicar_numeros}

llm_with_tools = llm.bind_tools([t for t in tools.values()])

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
messages = []
prompt = HumanMessage("Dame el resultado de multiplicar 8712387821763 y 12389749833")
messages.append(prompt)

# 1. Generar petición a Tool
ai_msg = llm_with_tools.invoke(messages)
print(f"ContentString: {ai_msg.content}")
print(f"ToolCalls: {ai_msg.tool_calls}")
messages.append(ai_msg)

# 2. Realizar petición a la Tool y guardar en ChatHistory
if ai_msg.tool_calls:
    for tool_call in ai_msg.tool_calls:
        tool_msg = tools[tool_call["name"]].invoke(tool_call)
        # print(f"ContentString: {tool_msg}")
        messages.append(tool_msg)
display(messages)

# 3. Repetir llamada, incluyendo salida de Tool
response = llm_with_tools.invoke(messages)
messages.append(response)
response

for msg in messages:
    msg.pretty_print()

In [ ]:
f"{8712387821763 * 12389749833:_}"  # Resultado correcto

Como el LLM tiene varias Tools, elegirá la más adecuada en cada ocasión 

In [ ]:
llm_with_tools.invoke("Dame el resultado de multiplicar 8712387821763 y 12389749833").tool_calls

In [ ]:
llm_with_tools.invoke("Quién es el presidente de Estados Unidos?").tool_calls

In [ ]:
llm_with_tools.invoke("Hola!").tool_calls

In [ ]:
llm_with_tools.invoke("Hola!").content

# Streaming

In [ ]:
print(llm.invoke("Crea un poema sobre LangChain").pretty_print())

In [ ]:
for chunk in llm.stream("Crea un poema sobre LangChain"):
    print(chunk.content, end="", flush=True)
